# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [ ]:
!git clone https://github.com/vladimirrim/TTS_HW
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


Cloning into 'TTS_HW'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 278 (delta 0), reused 1 (delta 0), pack-reused 273
Receiving objects: 100% (278/278), 4.73 MiB | 2.10 MiB/s, done.
Resolving deltas: 100% (69/69), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0  20382      0 --:--:-- --:--:-- --:--:-- 20382
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3283    0  3283    0     0  12675      0 --:--:-- --:--:-- --:--:-- 12724
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2254      0

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [ ]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8775, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 8775 (delta 1), reused 0 (delta 0), pack-reused 8765
Receiving objects: 100% (8775/8775), 14.44 MiB | 21.85 MiB/s, done.
Resolving deltas: 100% (5980/5980), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/iss

In [ ]:
% cd apex
! git reset --hard a651e2c
% cd ..

/content/apex
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/content


In [ ]:
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
Skipping wheel build for apex, due to binaries being disabled for it.
    Running setup.py install for apex ... done
/content


In [ ]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-813qbcis/dllogger_a5a03da4dbd04535abf678e803d50895
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-813qbcis/dllogger_a5a03da4dbd04535abf678e803d50895
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 12.7 MB/s 
     |████████████████████████████████| 195 kB 44.0 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=7a6ca972e997751a8ef9142c85a62269ef933ba8bad2124f39892a65ef2dedbe
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=li

In [ ]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-15 18:10:38.765156 - PARAMETER | output :  ./output
DLL 2021-12-15 18:10:38.765551 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-15 18:10:38.765690 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-15 18:10:38.765787 - PARAMETER | pyprof :  False
DLL 2021-12-15 18:10:38.765882 - PARAMETER | epochs :  10
DLL 2021-12-15 18:10:38.765956 - PARAMETER | epochs_per_ch

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-15 18:33:34.408568 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-15 18:33:34.408653 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-15 18:33:34.408728 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-15 18:33:34.408825 - PARAMETER | save_mels :  False
DLL 2021-12-15 18:33:34.408900 - PARAMETER | cuda :  True
DLL 2021-12-15 18:33:34.408984 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-15 18:33:34.409056 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-15 18:33:34.409120 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-15 18:33:34.409181 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-15 18:33:34.409241 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-15 18:33:34.409311 - PARAMETER | sampling_rate :  22050
DLL 2021-12-15 18:33:34.409387 - PARAMETER | stft_hop_length :  256
DLL 2021-12-15 18:33:34.409451 - PA

In [ ]:
import IPython
IPython.display.Audio(filename='TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [ ]:
!pwd

/content


In [ ]:
import IPython
IPython.display.Audio('TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [ ]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        trunc_labels = []
        for label in labels:
          trunc_label = label[:max_len]
          trunc_labels.append(trunc_label[:max_len] + 
                              [0] * (max_len - len(trunc_label)))

        self.labels = torch.tensor(trunc_labels).long()
        self.tokenizer = tokenizer
        self.max_len = max_len

        infos = tokenizer(sentences, return_tensors="pt", padding='max_length',
                          truncation=True, max_length=max_len)
        self.masks = infos.attention_mask.long()
        self.ids = infos.input_ids.long()
        
    def __getitem__(self, index):
        return {
            'input_ids': self.ids[index],
            'attention_mask': self.masks[index],
            'labels': self.labels[index]
        } 
    
    def __len__(self):
        return self.len

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
sentences = []
label_proms = []
label_boundaries = []

for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    with open(filename, 'r') as f:
        sentence, prominence  = [], [0]
        for line in f.readlines():
            args = line.split('\t')
            sentence.append(args[-3])
            prominence.extend([val_to_class(float(args[-2]))] * 
                              len(tokenizer(args[-3], add_special_tokens=False).input_ids))
        
        
        sentences.append(' '.join(sentence))
        label_proms.append(prominence)


train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()
        out = model(**{k: v.to(device) for k, v in batch.items()})
        out.loss.backward()
        t_bar.set_description(desc=f'loss; {out.loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            out = model(**{k: v.to(device) for k, v in batch.items()})

            y_hat = out.logits.cpu().argmax(-1)[batch["attention_mask"] == 1]
            target = batch["labels"][batch["attention_mask"] == 1]

            accs.append(acc_criterion(y_hat, target))
            f1_scores.append(f1_criterion(y_hat, target))

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

EPOCH:0


loss; 0.6358126401901245: 100%|██████████| 14/14 [00:22<00:00,  1.58s/it]


Accuracy: 0.7415730357170105
F1 score: 0.7415730357170105
EPOCH:1


loss; 0.5660350918769836: 100%|██████████| 14/14 [00:22<00:00,  1.58s/it]


...

Accuracy: 0.7528089880943298
F1 score: 0.7528089880943298
EPOCH:8


loss; 0.1847393959760666: 100%|██████████| 14/14 [00:22<00:00,  1.57s/it]


Accuracy: 0.7565543055534363
F1 score: 0.7565543055534363
EPOCH:9


loss; 0.11715427786111832: 100%|██████████| 14/14 [00:21<00:00,  1.57s/it]


Accuracy: 0.7565543055534363
F1 score: 0.7565543055534363


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!